In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import subprocess

try:
    import japanize_matplotlib
    print("japanize_matplotlib is already installed.")
except ImportError:
    !pip install japanize_matplotlib
    print("japanize_matplotlib has been installed.")

try:
    import catboost
    print("catboost is already installed.")
except ImportError:
    !pip install catboost
    print("catboost has been installed.")

try:
    import optuna
    print("optuna is already installed.")
except ImportError:
    !pip install optuna
    print("optuna has been installed.")

try:
    import geolib
    print("geolib is already installed.")
except ImportError:
    !pip install geolib
    print("geolib has been installed.")

# mkdirとechoコマンドの実行
try:
    subprocess.check_call('mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd', shell=True)
    print('Directory and file for OpenCL vendors have been created.')
except subprocess.CalledProcessError as e:
    print(f"Error running mkdir and echo command: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=99de94043136a552b1d24db46ec00da84cf670dd22ce222f79af1e581498bd06
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib
japanize_matplotlib has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 26.3 MB/s eta 0:00:00
catboost has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00
optuna has been installed.
geolib has been installed.
Directory and file for OpenCL vendors have been created.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns
import pickle

import sys
sys.path.append('/content/drive/MyDrive/yuto/project_directory/code')

from utils_ver2 import (
                   to_cat_col,
                   VotingModel,
                   check_dataframe_elements
                   )

TOTAL_PRICE_LOG="取引価格（総額）_log"
PRICE_PER_AREA_LOG="単位面積あたりの取引価格_log"

model_cat_lq_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_cat_lq_cv_20240908_141325_0_06609571831620502.pkl"
model_lgbm_l1_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_20240908_110735_0_06585289841.pkl"
model_lgbm_l1_cv_target_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_target_20240910_144521_0_06636916360273092.pkl"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
def visualize_hyperparameters(estimators):
    if isinstance(estimators, VotingModel):  # 入力がVotingModelかどうかを確認します
        for estimator in estimators.estimators:  # VotingModel内の推定器を反復処理します
            params = estimator.get_params()
            print(f"Estimator: {estimator.__class__.__name__}")  # 推定器のタイプを表示します
            #for param, value in params.items():
            #    print(f"  {param}: {value}")
            print(params)
    else:
        params = estimators.get_params()
        print(f"Estimator: {estimators.__class__.__name__}")  # 推定器のタイプを表示します
        #for param, value in params.items():
        #    print(f"  {param}: {value}")
        print(params)

# モデルの読み込み
def read_model(model_path):
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    return model

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/preprocess_main_table.csv",index_col=0)
#df=df.drop(columns=["住所_地区名欠損フラグ","address","title",PRICE_PER_AREA_LOG],axis=1)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log', '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]

In [ ]:
df

,dataset,ID,価格情報区分,都道府県名,市区町村名,地区名,住所不一致_Flag,住所,最寄駅：名称,最寄駅：距離（分）,...,県_市_geohash_3_駅徒歩時間平均,県_市_geohash_3_建築年平均,県_市_geohash_4_駅徒歩時間平均,県_市_geohash_4_建築年平均,県_市_geohash_5_駅徒歩時間平均,県_市_geohash_5_建築年平均,県_市_geohash_6_駅徒歩時間平均,県_市_geohash_6_建築年平均,県_市_geohash_7_駅徒歩時間平均,県_市_geohash_7_建築年平均
0,train,35007328,不動産取引価格情報,山口県,下関市,秋根西町,0,山口県下関市秋根西町,新下関,2.0,...,24.497130,1997.324232,24.497130,1997.324232,12.088968,1999.057143,4.848485,2001.266667,4.142857,2000.846154
1,train,35048345,不動産取引価格情報,山口県,周南市,五月町,0,山口県周南市五月町,徳山,45.0,...,16.293403,1994.935043,15.674336,1994.977233,15.986842,1994.156522,29.200000,1997.000000,29.200000,1997.000000
2,train,35003676,不動産取引価格情報,山口県,下関市,山の田東町,0,山口県下関市山の田東町,幡生,14.0,...,24.497130,1997.324232,24.497130,1997.324232,12.088968,1999.057143,11.514286,2001.228571,11.565217,2001.869565
3,train,35053427,成約価格情報,山口県,山陽小野田市,大字山川,0,山口県山陽小野田市大字山川,厚狭,3.0,...,3.000000,1997.000000,3.000000,1997.000000,3.000000,2001.000000,3.000000,2001.000000,3.000000,2001.000000
4,train,35005127,不動産取引価格情報,山口県,下関市,一の宮町,0,山口県下関市一の宮町,新下関,2.0,...,24.497130,1997.324232,24.497130,1997.324232,12.088968,1999.057143,6.800000,1993.476190,10.538462,2006.592593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063668,test,47036620,成約価格情報,沖縄県,中頭郡北中城村,字ライカム,0,沖縄県中頭郡北中城村字ライカム,てだこ浦西,NaN,...,NaN,2017.727273,NaN,2017.727273,NaN,2017.444444,NaN,2017.444444,NaN,2017.444444
1063669,test,47037115,成約価格情報,沖縄県,中頭郡中城村,字南上原,0,沖縄県中頭郡中城村字南上原,てだこ浦西,NaN,...,NaN,2012.250000,NaN,2012.250000,NaN,2012.250000,NaN,2012.250000,NaN,2012.250000
1063670,test,47037850,成約価格情報,沖縄県,中頭郡西原町,字翁長,0,沖縄県中頭郡西原町字翁長,てだこ浦西,NaN,...,8.000000,2012.600000,8.000000,2012.600000,8.000000,2005.500000,8.000000,2005.500000,8.000000,2005.500000
1063671,test,47037855,成約価格情報,沖縄県,中頭郡西原町,上原,0,沖縄県中頭郡西原町上原,てだこ浦西,NaN,...,8.000000,2012.600000,8.000000,2012.600000,NaN,2013.692308,NaN,2013.692308,NaN,2013.692308


In [ ]:
# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

print("Train DataFrame:")
print(df_train)
print("\nTest DataFrame:")
print(df_test)

Train DataFrame:
             価格情報区分 都道府県名   市区町村名    地区名  住所不一致_Flag             住所 最寄駅：名称  \
ID                                                                           
35007328  不動産取引価格情報   山口県     下関市   秋根西町           0     山口県下関市秋根西町    新下関   
35048345  不動産取引価格情報   山口県     周南市    五月町           0      山口県周南市五月町     徳山   
35003676  不動産取引価格情報   山口県     下関市  山の田東町           0    山口県下関市山の田東町     幡生   
35053427     成約価格情報   山口県  山陽小野田市   大字山川           0  山口県山陽小野田市大字山川     厚狭   
35005127  不動産取引価格情報   山口県     下関市   一の宮町           0     山口県下関市一の宮町    新下関   
...             ...   ...     ...    ...         ...            ...    ...   
6018727   不動産取引価格情報   山形県     酒田市     幸町           0       山形県酒田市幸町     酒田   
6001936   不動産取引価格情報   山形県     山形市    十日町           0      山形県山形市十日町     山形   
6008180   不動産取引価格情報   山形県     山形市   小荷駄町           0     山形県山形市小荷駄町     山形   
6006734   不動産取引価格情報   山形県     山形市    相生町           0      山形県山形市相生町     山形   
6009259   不動産取引価格情報   山形県     山形市    六日町       

In [ ]:
#check_dataframe_elements(df)

In [ ]:
def decode_price(y_pred,df_test):
    # 単位面積あたりの取引価格の対数から10のべき乗を取る
    price_per_area = 10**y_pred
    # その値に面積を掛けて、取引価格（総額）を計算
    predicted_price_total = price_per_area * df_test["面積（㎡）"]
    # 取引価格（総額）の対数を計算
    y_pred= np.log10(predicted_price_total)
    #y_pred=y_pred.rename(columns={"面積（㎡）":"取引価格（総額）_log"})
    y_pred.name = "取引価格（総額）_log"
    return y_pred

In [ ]:
#print(abc)

Catboost l1

LightGBM l1 CV

LightGBM l1 CV target

In [ ]:
model_lgbm_l1_cv_target=read_model(model_lgbm_l1_cv_target_path)

In [ ]:
visualize_hyperparameters(model_lgbm_l1_cv_target)

#y_pred_lgbm_l1_cv_target=pd.Series(model_lgbm_l1_cv_target.predict_lgbm(df_test.drop(TOTAL_PRICE_LOG,axis=1)),index=df_test.index)
y_pred_lgbm=pd.Series(model_lgbm_l1_cv_target.predict_lgbm(df_test.drop(TOTAL_PRICE_LOG,axis=1)),index=df_test.index)
#y_pred_lgbm=decode_price(y_pred_lgbm,df_test)
# Seriesの名前を変更
y_pred_lgbm.name = "取引価格（総額）_log"

# y_predをデータフレームに変換
df_pred = pd.DataFrame(y_pred_lgbm, columns=["取引価格（総額）_log"])

# 結果を表示
print(df_pred)

df_pred.to_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/test_lgbm_cv_target.csv")

Estimator: LGBMRegressor
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.25115722950204683, 'importance_type': 'split', 'learning_rate': 0.04475736155212619, 'max_depth': 146, 'min_child_samples': 594, 'min_child_weight': 8.279252981359091, 'min_split_gain': 0.7386917462339693, 'n_estimators': 48260, 'n_jobs': None, 'num_leaves': 31, 'objective': 'l1', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 0.6115829632198598, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'metrics': 'mae', 'boost_from_average': False, 'verbose': -1, 'colsample_bynode': 0.23761739079852606, 'bagging_fraction': 0.9163657118215358, 'feature_fraction': 0.5366140292624185, 'max_bin': 1728, 'epsilon': 7.743499163314079, 'alpha': 0.7814548201134592, 'min_sum_hessian_in_leaf': 0.05058971217721564}
Estimator: LGBMRegressor
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.25115722950204683, 'importance_type': 'split', 'learning_rate': 0.044757361552126

In [5]:
df_sub=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/test_l1_tg20240910_144521.csv")
df_sub

,ID,取引価格（総額）_log
0,1000107,6.775977
1,1000109,7.321303
2,1000112,6.670467
3,1000114,6.476497
4,1000117,7.405344
...,...,...
54397,47036620,7.534591
54398,47037115,7.455753
54399,47037850,7.256834
54400,47037855,7.519464
